## Feature Engeneering

Este notebook contém a etapa de feature engineering dos conjuntos de treino e teste. Essa etapa se baseia no pré-processamento da etapa de Exploratory Analisys, realizada anteriormente.

Ao final dessa etapa, serão gerados dois conjuntos de dados de treino e teste processados, que serão utilizados para treinamento e teste dos modelos de classificação.

#### Importação de bibliotecas:

In [69]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import * 
from nltk import word_tokenize as TK

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import nltk
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Importação dos datasets:

In [70]:
# read datasets
train = pd.read_csv('dataset/train.csv', encoding='utf-8')
test = pd.read_csv('dataset/test.csv', encoding='utf-8')

In [71]:
print("Existem {} exemplo, no dataset de treinamento. Cada exemplo contém {} atributos, sendo: "
      .format(train.shape[0],train.shape[1]) + ", ".join(train.columns) + '.')

Existem 20800 exemplo, no dataset de treinamento. Cada exemplo contém 5 atributos, sendo: id, title, author, text, label.


In [72]:
# visualização das 5 primeiras notícias do conjunto de treino
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [73]:
print("Existem {} exemplo, no dataset de treinamento. Cada exemplo contém {} atributos, sendo: "
      .format(test.shape[0],test.shape[1]) + ", ".join(train.columns) + '.')

Existem 5200 exemplo, no dataset de treinamento. Cada exemplo contém 4 atributos, sendo: id, title, author, text, label.


In [74]:
# visualização das 5 primeiras notícias do conjunto de teste
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


### Tratamento dos dados
Nessa etapa serão realizados alguns tratamentos nos dados de treinamento e teste:
* Tratamento de missing values
* Conversão de todas as palavras para letras minúsculas
* Remoção de caracteres númericos e especiais
* Remoção de stopwords
* Lematização das palavras

##### Tratamento de missing values e outros exemplos que precisam ser removidos

In [75]:
# remover exemplos com string nula
train = train[~train['text'].isnull()]
test = test[~test['text'].isnull()]

# preenche títulos nulos com string vazia
train['title'].fillna('',inplace=True)
test['title'].fillna('',inplace=True)

In [76]:
import unicodedata as ud

latin_letters= {}

def is_latin(uchr):
    try: return latin_letters[uchr]
    except KeyError:
         return latin_letters.setdefault(uchr, 'LATIN' in ud.name(uchr))

def only_roman_chars(unistr):
    return all(is_latin(uchr)
           for uchr in unistr
           if uchr.isalpha()) # isalpha suggested by John Machin

In [77]:
# remover exemplos com texto em outros alfabetos
train['only_roman_chars'] = train['text'].apply(only_roman_chars)
train = train[train['only_roman_chars']==True]
train.shape

(20478, 6)

In [78]:
test['only_roman_chars'] = train['text'].apply(only_roman_chars)
test = test[test['only_roman_chars']==True]
test.shape

(5112, 5)

##### Concatenação do título com o texto

Para que o título seja considerado nas análises, as strings serão concatenadas e serão analisadas em conjunto

In [79]:
train['title_text'] = train['title'] + ' ' + train['text']
test['title_text'] = test['title'] + ' ' + test['text']

##### Conversão para minúsculas

In [80]:
train['processed_text'] = train['title_text'].str.lower()
test['processed_text'] = test['title_text'].str.lower()

##### Tokenização de texto e título

In [81]:
train['processed_text'] = train['processed_text'].apply(lambda x:list(nltk.word_tokenize(x)))
test['processed_text'] = test['processed_text'].apply(lambda x:list(nltk.word_tokenize(x)))

##### Remoção de números, caracteres especiais
É necessário realizar uma limpeza no texto para igualar palavras iguais que podem ter sido escritas de forma incorreta ou diferente, palavras que não tem sentido sozinhos como números. Para isso faremos:
- remoção de caracteres numéricos
- remoção de caracteres especiais e pontuação
- remoção de acentos, hífens, apóstrofo
- remoção de stopwords
- stemming
- lematização

In [82]:
# retira os caracteres numéricos 
train['processed_text'] = train['processed_text'].apply(lambda x: [re.sub('[0-9]', '',i) for i in x])
test['processed_text'] = test['processed_text'].apply(lambda x: [re.sub('[0-9]', '',i) for i in x])

In [83]:
# retira os caracteres especiais e pontuação
train['processed_text'] = train['processed_text'].apply(lambda x: [re.sub('[?_!%&/.,®€™():]', '',i) for i in x])
train['processed_text'] = train['processed_text'].apply(lambda x: [i.replace("\\r", "").replace("\\n","") for i in x])

test['processed_text'] = test['processed_text'].apply(lambda x: [re.sub('[?_!%&/.,®€™():]', '',i) for i in x])
test['processed_text'] = test['processed_text'].apply(lambda x: [i.replace("\\r", "").replace("\\n","") for i in x])

In [84]:
# remove hífen e apóstrofo, adicionando espaço no lugar
train['processed_text'] = train['processed_text'].apply(lambda x: [re.sub('[-“”[—‘’\']', ' ',i) for i in x])
test['processed_text'] = test['processed_text'].apply(lambda x: [re.sub('[-“”[—‘’\']', ' ',i) for i in x])

In [85]:
# remoção de acentos
train['processed_text'] = train['processed_text'].apply(lambda x: [i.replace('â', "a").replace('è', 'e').replace('é', 'e').replace('í', 'i').replace('î', 'i').replace('ú', 'u').replace('ç', 'c') for i in x])
test['processed_text'] = test['processed_text'].apply(lambda x: [i.replace('â', "a").replace('è', 'e').replace('é', 'e').replace('í', 'i').replace('î', 'i').replace('ú', 'u').replace('ç', 'c') for i in x])

##### Remoção de stop words

In [86]:
# lista de stopwords em inglês
stop_words = set(stopwords.words('english')) 

train['processed_text'] = train['processed_text'].apply(lambda x: [item for item in x if item not in stop_words])
test['processed_text'] = test['processed_text'].apply(lambda x: [item for item in x if item not in stop_words])

##### Lematização

In [87]:
lemmatizer = nltk.stem.WordNetLemmatizer()
train['processed_text'] = train['processed_text'].apply(lambda x:  [lemmatizer.lemmatize(w) for w in x])
test['processed_text'] = test['processed_text'].apply(lambda x:  [lemmatizer.lemmatize(w) for w in x])

##### Stemming

In [88]:
stemmer = PorterStemmer()  
train['processed_text'] = train['processed_text'].apply(lambda x:  [stemmer.stem(i) for i in x])
test['processed_text'] = test['processed_text'].apply(lambda x:  [stemmer.stem(i) for i in x])

##### Remoção de palavras vazias (espaços em branco)

In [89]:
train['processed_text'] = train['processed_text'].apply(lambda x: [a.strip() for a in x if a.strip()])
test['processed_text'] = test['processed_text'].apply(lambda x: [a.strip() for a in x if a.strip()])

##### Exclusão das notícias repetidas

In [90]:
# cria variável de texto, com palavras separadas por vírgula
train["words_text"] = train["processed_text"].apply(lambda x: ", ".join(x))
test["words_text"] = test["processed_text"].apply(lambda x: ", ".join(x))

# removendo casos duplicados
train.drop_duplicates(subset=['words_text'], keep='first', inplace=True)
test.drop_duplicates(subset=['words_text'], keep='first', inplace=True)

### Criação de novas features

Uma feature foi criada na etapa anterior e contém: a lista de palavras de cada notícia concatenadas em apenas uma string, separadas por vírgula.
Com base na coluna de texto e título processado, será criada, para cada coluna, uma nova feature com a quantidade de palavras de cada texto e cada título

In [91]:
train['processed_word_count'] = train['processed_text'].apply(lambda x: len(x))
test['processed_word_count'] = test['processed_text'].apply(lambda x: len(x))

In [92]:
# removendo textos que ficaram vazios após o tratamento e limpeza
train = train[train['processed_word_count']!=0]

### Datasets após o tratamento

In [93]:
print("O conjunto de treinamento processado possui {} exemplos e \
{} colunas:".format(train.shape[0],train.shape[1])) 

O conjunto de treinamento processado possui 20339 exemplos e 10 colunas:


In [94]:
# visualização das cinco primeira notícias presentes no conjunto de treino
train.head()

,id,title,author,text,label,only_roman_chars,title_text,processed_text,words_text,processed_word_count
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,True,House Dem Aide: We Didn’t Even See Comey’s Let...,"[hous, dem, aid, even, see, comey, letter, jas...","hous, dem, aid, even, see, comey, letter, jaso...",445
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,True,"FLYNN: Hillary Clinton, Big Woman on Campus - ...","[flynn, hillari, clinton, big, woman, campu, b...","flynn, hillari, clinton, big, woman, campu, br...",374
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,True,Why the Truth Might Get You Fired Why the Trut...,"[truth, might, get, fire, truth, might, get, f...","truth, might, get, fire, truth, might, get, fi...",697
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,True,15 Civilians Killed In Single US Airstrike Hav...,"[civilian, kill, singl, u, airstrik, identifi,...","civilian, kill, singl, u, airstrik, identifi, ...",309
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,True,Iranian woman jailed for fictional unpublished...,"[iranian, woman, jail, fiction, unpublish, sto...","iranian, woman, jail, fiction, unpublish, stor...",99


In [95]:
print("O conjunto de teste processado possui {} exemplos e \
{} colunas:".format(test.shape[0],test.shape[1])) 

O conjunto de teste processado possui 5106 exemplos e 9 colunas:


In [96]:
# visualização das cinco primeira notícias presentes no conjunto de treino
test.head()

,id,title,author,text,only_roman_chars,title_text,processed_text,words_text,processed_word_count
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",True,"Specter of Trump Loosens Tongues, if Not Purse...","[specter, trump, loosen, tongu, purs, string, ...","specter, trump, loosen, tongu, purs, string, s...",776
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,True,Russian warships ready to strike terrorists ne...,"[russian, warship, readi, strike, terrorist, n...","russian, warship, readi, strike, terrorist, ne...",159
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,True,#NoDAPL: Native American Leaders Vow to Stay A...,"[#, nodapl, nativ, american, leader, vow, stay...","#, nodapl, nativ, american, leader, vow, stay,...",447
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",True,"Tim Tebow Will Attempt Another Comeback, This ...","[tim, tebow, attempt, anoth, comeback, time, b...","tim, tebow, attempt, anoth, comeback, time, ba...",365
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,True,Keiser Report: Meme Wars (E995) 42 mins ago 1 ...,"[keiser, report, meme, war, e, min, ago, view,...","keiser, report, meme, war, e, min, ago, view, ...",57


## TF-IDF e exportação dos datasets processados

#### Salvar os valores de target e ids em um novo arquivo

In [97]:
train = train.reset_index(drop=True)
y_train = train['label']
pd.DataFrame(y_train, columns=['target']).to_csv("dataset/train_target.csv", index=False)

In [98]:
index_train = list(train.index)
ids_train = train['id']
d = {'index': index_train, 'id': ids_train}
pd.DataFrame(d).to_csv("dataset/train_ids.csv", index=False)

#### TF-IDF

Como os modelos preditivos não trabalham diretamente com textos, é necessário realizar a vetorização da lista de palavras das notícias. Será utilizado o método TF-IDF para isso:

In [99]:
def tf_idf(df):
    df = df.reset_index(drop=True)
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(df['words_text'])
    #print(vectorizer.get_feature_names())
    print(tfidf.shape)
    return tfidf

In [100]:
df=train
tfidf_train = tf_idf(df)
X_train = tfidf_train
scipy.sparse.save_npz('dataset/processed_train.npz', X_train)

(20339, 115846)


In [101]:
tfidf_test = tf_idf(test)
X_test = tfidf_test
scipy.sparse.save_npz('dataset/processed_test.npz', X_test)

(5106, 66718)
